<h1> Algorithmen in der Algebra, Woche 8 </h1>
<h2> Multivariate Polynome in Sage</h2>
Wir können multivariate Polynomringe wie folgt definieren. $\verb|order="lex"|$ weist SageMath an, die Monome lexikografisch zu ordnen; wenn es weggelassen wird, wird es eine andere Wohlordnung für die Monome wählen (die sogenannte "degree reverse lexicographic order", die in der Vorlesung nicht behandelt wurde).

In [1]:
R.<x1,x2,x3,x4> = PolynomialRing(QQ,order="lex")

$\verb|f.monomials()|$ gibt die Liste der Monome zurück, lexikographisch geordnet.

$\verb|f.coefficients()|$ gibt die Liste der Koeffiziente, in der gleichen Reihenfolge.

In [2]:
f=x1*x4+7*x2*x3+2*x1
print(f.monomials())
print(f.coefficients())

[x1*x4, x1, x2*x3]
[1, 2, 7]


">" vergleicht Monome in Bezug auf die gewählte Monomordnung (und <,<=,>= ähnlich).

In [3]:
x1*x4>x2*x3

True

Verwenden Sie .divides(), um zu testen, ob ein Monom durch ein anderes teilbar ist.

In [4]:
x1.divides(x1*x2)

True

Um die Monome dann zu teilen, verwenden Sie "//" (und nicht "/", dann behandelt Sage es als rationale Funktion anstelle eines Polynoms).

In [5]:
(x1*x2)//x1

x2

<h2> Aufgabe </h2>
Implementieren Sie die Funktion Teilung.

Sie können den folgenden Code verwenden, um Ihre Funktion zu testen (wenn Sie möchten, können Sie f und h durch andere Polynome ersetzen). Wenn es ausgeführt wird, ohne dass ein "Assertion Error" ausgelöst wird, bedeutet dies, dass das Ergebnis korrekt ist.

In [6]:
f=x1^3*x3^2
h=[x1^2+x1*x2^2,x1^2*x3+x4]
(r,q)=teilung(f,h)
print(r)
print(q)
k=len(h)
assert(sum([q[i]*h[i] for i in range(k)])+r==f)
for hi in h:
    for mon in r.monomials():
        assert not hi.monomials()[0].divides(mon)
for i in range(k):
    assert q[i]==0 or (q[i]*h[i]).monomials()[0] <= f.monomials()[0]

NameError: name 'teilung' is not defined

In [13]:
f.factor()

x3^2 * x1^3